# CONJECTURES

In this notebook we define the functions that ultimately test our conjectures.

Some functions share a lot of code. It is possible to speed up the code by checking multiple conjectures at once. We opted to sacrifice speed for comprehensibility.

In [1]:
#IMPORTS
#libraries
import numpy as np
import itertools
import time
import import_ipynb

#notebooks
import blockchain as bc
import helper_functions as helfun
import payoff_matrix as pm

importing Jupyter notebook from blockchain.ipynb
importing Jupyter notebook from helper_functions.ipynb
importing Jupyter notebook from payoff_matrix.ipynb


## CONJECTURE

In [2]:
def Switching(_T, _n):
    #
    #check if there are situations where a miner ever switches to a shorter branch
    #for this purpose this function generates EVERY chain of lengths 1,2, ..., T-1 that is possible in a '_T' stage game, computes both miner's strategies,
    #and compares them to their strategies in the subsequent stage
    #
      
    for t in range(1, _T-1):

        #GENERATE POSSIBLE BLOCKCHAINS
        possible_parents = helfun.generatePossibleParents(t)
        possible_winners = helfun.generatePossibleWinners(t, _n)

        #FOR ALL BLOCKCHAINS OF HEIGHT 't'
        for parents in possible_parents:
            for winners in possible_winners:
                B = bc.Blockchain(_n, parents, winners)

                M = pm.intermediatePayoffMatrix(B, _T, _n)

                nash = pm.matrixNashEq(M) #get the equilibria
                nash_strats = np.transpose(np.where(nash == True)) #get the strategy profiles
                nash_strats = np.array([np.array(element) for element in nash_strats]) #convert to array

                for strat in nash_strats: #for every possible Nash equilibrium

                    B_ext0 = bc.ExtendedBlockchain(B, 0, strat[0]) #extend the chain if miner 0 wins
                    B_ext1 = bc.ExtendedBlockchain(B, 1, strat[1]) #extend the chain if miner 1 wins

                    M_ext0 = pm.intermediatePayoffMatrix(B_ext0, _T, _n) #payoff matrix if miner 0 wins
                    M_ext1 = pm.intermediatePayoffMatrix(B_ext1, _T, _n) #payoff matrix if miner 1 wins

                    #GET STRATEGIES FOR BOTH CASES
                    nash0 = pm.matrixNashEq(M_ext0) #case where miner 0 wins
                    nash_strats0 = np.transpose(np.where(nash0 == True))
                    nash_strats0 = np.array([np.array(element) for element in nash_strats0])
                    nash1 = pm.matrixNashEq(M_ext1) #case where miner 1 wins
                    nash_strats1 = np.transpose(np.where(nash1 == True))
                    nash_strats1 = np.array([np.array(element) for element in nash_strats1])

                    length_before_m0 = B.chainLength(strat[0]) #length of chain where miner 0 mined at in stage 't'
                    length_before_m1 = B.chainLength(strat[1]) #length of chain where miner 1 mined at in stage 't'

                    #COMPARE THE LENGTH OF THE CHAINS
                    for strat0 in nash_strats0: #the case where miner 0 won
                        if B_ext0.chainLength(strat0[0]) < length_before_m0: #comparison for miner 0
                            print("Found a counter-example!")
                            return 0
                        if B_ext0.chainLength(strat0[1]) < length_before_m1: #comparison for miner 1
                            print("Found a counter-example!")
                            return 0
                    for strat1 in nash_strats1: #the case where miner 1 won
                        if B_ext1.chainLength(strat1[0]) < length_before_m0: #comparison for miner 0
                            print("Found a counter-example!")
                            return 0
                        if B_ext1.chainLength(strat1[1]) < length_before_m1: #comparison for miner 1
                            print("Found a counter-example!")
                            return 0

        print(f"checked all situations for t = {t+1} and  T = {_T}") #print status

## CONJECTURE

In [3]:
def FirstBlock(_T, _n):
    #
    #check if there are situations where a miner ever switches away from the branch they won their first block on
    #for this purpose this function generates EVERY chain where ONLY ONE miner won any blocks on of heights 1,2, ..., T-1 that are possible in a '_T' stage game, 
    #then computes the other miner's strategy, iterates through the game according to the equilibrium strategies and checks after each stage if the conjecture is violated
    #
      
    for t in range(1, _T-1):

        #GENERATE POSSIBLE BLOCKCHAINS
        possible_parents = helfun.generatePossibleParents(t)
        winners = np.zeros(t) #generate a history where only one miner (w.l.o.g. miner 0) won any rounds

        #FOR ALL BLOCKCHAINS OF HEIGHT 't'
        for parents in possible_parents:
            B = bc.Blockchain(_n, parents, winners)
           
            M = pm.intermediatePayoffMatrix(B, _T, _n)

            nash = pm.matrixNashEq(M) #get the equilibria
            nash_strats = np.transpose(np.where(nash == True)) #get the strategy profiles
            nash_strats = np.array([np.array(element) for element in nash_strats]) #convert to array

            for strat in nash_strats: #for every possible Nash equilibrium
                found = recursionFirstBlock(B, _T, 1, strat[1], B.horizon + 1) #call recursion when miner 1 wins their first block
                if found == True:
                    print("Found counter-example!")
                    return 0

        print(f"checked all situations for t = {t+1} and  T = {_T}") #print status



def recursionFirstBlock(_B, _T, _i, _t, _c):        
    #
    #'_B' is the blockchain of the previous stage
    #'_T' is the number of stages of the game
    #'_i' is the index of the winning miner of the previous stage
    #'_t' is the index of the block that winner chose
    #'_c' is the index of the first block miner 1 won.
    #

    B_ext = bc.ExtendedBlockchain(_B, _i, _t) #extend the chain if mi won

    if B_ext.horizon == _T: #we have reached the end of the game
        return False

    M_ext = pm.intermediatePayoffMatrix(B_ext, _T, 2)

    nash = pm.matrixNashEq(M_ext) #get a mask of nash equilibria
    nash_strats = np.transpose(np.where(nash == True)) #get the strategy profiles
    nash_strats = np.array([np.array(element) for element in nash_strats]) #convert to np.array

    found = False
    for strat in nash_strats: #for every possible Nash equilibrium

        #CHECK IF WE FOUND A COUNTER-EXAMPLE
        found = not helfun.isOnSameBranch(B_ext, _c, strat[1])
        if found == True:
            break
        else: #else, go deeper
            found = recursionFirstBlock(B_ext, _T, 0, strat[0], _c) #the case where miner 0 wins
            if found == True:
                break
            found = recursionFirstBlock(B_ext, _T, 1, strat[1], _c) #the case where miner 1 wins
            if found == True:
                break

    return found

## CONJECTURE

In [4]:
def Monotonicity(_T, _n):
    #
    #check if there are situations where monotonicity is violated
    #for this purpose this function generates EVERY chain of heights 1,2, ..., T-1 that is possible in a '_T' stage game, then computes both miner's strategies,
    #and compares them to their strategies in the subsequent stage      
    #

    for t in range(1, _T-1):

        #GENERATE POSSIBLE BLOCKCHAINS
        possible_parents = helfun.generatePossibleParents(t)
        possible_winners = helfun.generatePossibleWinners(t, _n)

        #FOR ALL BLOCKCHAINS OF HEIGHT 't'
        for parents in possible_parents:
            for winners in possible_winners:
                B = bc.Blockchain(_n, parents, winners)

                M = pm.intermediatePayoffMatrix(B, _T, _n)

                nash = pm.matrixNashEq(M) #get the equilibria
                nash_strats = np.transpose(np.where(nash == True)) #get the strategy profiles
                nash_strats = np.array([np.array(element) for element in nash_strats]) #convert to array

                for strat in nash_strats: #for every possible Nash equilibrium

                    B_ext0 = bc.ExtendedBlockchain(B, 0, strat[0]) #extend the chain if miner 0 won
                    B_ext1 = bc.ExtendedBlockchain(B, 1, strat[1]) #extend the chain if miner 1 won

                    M_ext0 = pm.intermediatePayoffMatrix(B_ext0, _T, _n) #payoff matrix if miner 0 won
                    M_ext1 = pm.intermediatePayoffMatrix(B_ext1, _T, _n) #payoff matrix if miner 1 won

                    #GET STRATEGIES FOR BOTH CASES
                    nash0 = pm.matrixNashEq(M_ext0) #case where miner 0 won
                    nash_strats0 = np.transpose(np.where(nash0 == True))
                    nash_strats0 = np.array([np.array(element) for element in nash_strats0])
                    nash1 = pm.matrixNashEq(M_ext1) #case where miner 1 won
                    nash_strats1 = np.transpose(np.where(nash1 == True))
                    nash_strats1 = np.array([np.array(element) for element in nash_strats1])

                    #COMPARE STRATEGIES
                    #see if any miner keeps mining on the same block in the next stage 't+1'
                    #also check if the most recent block is appended to the block the miner mined at in the previous stage 't', only then are we in a situation where monotonicity could be violated
                    for strat0 in nash_strats0: #the case where miner 0 won
                        if strat0[0] == strat[0] and strat0[0] == B_ext0.sequence[-1].parent: #check for miner 0
                            print("Found a counter-example! 1")
                            return 0
                        if strat0[1] == strat[1] and strat0[1] == B_ext0.sequence[-1].parent: #check for miner 1
                            print("Found a counter-example! 2")
                            return 0
                    for strat1 in nash_strats1: #the case where miner 1 won
                        if strat1[0] == strat[0] and strat1[0] == B_ext1.sequence[-1].parent: #check for miner 0
                            print("Found a counter-example! 3")
                            return 0
                        if strat1[1] == strat[1] and strat1[1] == B_ext1.sequence[-1].parent: #check for miner 1
                            print("Found a counter-example! 4")
                            return 0

        print(f"checked all situations for t = {t+1} and  T = {_T}") #print status

## CONJECTURE

In [5]:
def Forks(_T, _n):
    #
    #'_T' is the number of stages in the game
    #'_n' is the number of players, which must be 2
    #
    #we consider on-path situations
    #we want to figure out if there is an on-path situation where at least one miner does not mine on the most recent block. That would mean that they create a fork
    #

    assert _T > 2, "go further!"

    #BUILD BASE CHAIN
    parents = np.array([0])
    winners = np.array([0]) #miner 0 wins the first stage w.l.o.g.

    B = bc.Blockchain(_n, parents, winners)

    nash_strats = pm.getStrategies(B, _T, _n)

    if len(nash_strats) > 1: #check if there are multiple equilibria
        print('multiple equilibrium strategies found!')
        return True #counter-example with more than one nash-eq (on-path) found
    for i in range(_n):
        B_ext = bc.ExtendedBlockchain(B, i, nash_strats[0][i]) #extend the blockchain in the case where miner 'i' wins
        found = recursionForks(B_ext, 2, _T, _n) #start in the second stage
        if found == True:
            return True
  
    return False



def recursionForks(_B, _t, _T, _n):
    #
    #'_B' is the blockchain of the previous stage
    #'_t' is the current stage
    #'_T' is the number of stages in the game
    #'_n' is the number of miners
    #

    t = _t + 1

    nash_strats = pm.getStrategies(_B, _T, _n)

    if len(nash_strats) > 1: #check if there are multiple equilibria
        print('multiple equilibrium strategies found!')
        return True #counter-example with more than one nash-eq (on-path) found
    if t == _T:
        return False #no counter-example found
    for i in range(_n):
        B_ext = bc.ExtendedBlockchain(_B, i, nash_strats[0][i]) #extend the blockchain in the case where miner 'i' wins
        found = recursionForks(B_ext, t, _T, _n) #found a counter-example?
        if found == True:
            return True

    return False 

## CONJECTURE
[insert notes here]

In [6]:
def Assumption(_T, _n):
    #
    #checks if there are any situations where our assumption in 'intermediatePayoffMatrix()' is a problem
    #for this purpose it generates EVERY chain of lengths t=1,2,...,T that is possible in a T stage game, then computes the payoff matrix for each,
    #and checks if we have a situation where our assumption is absurd
    #see if coordination is ever necessary to play a Nash equilibrium
    #
      
    for t in range(1, _T):

        #GENERATE POSSIBLE BLOCKCHAINS
        possible_parents = helfun.generatePossibleParents(t)
        possible_winners = helfun.generatePossibleWinners(t, _n)
        
        #FOR ALL BLOCKCHAINS OF HEIGHT 't'
        for parents in possible_parents:
            for winners in possible_winners:
                B = bc.Blockchain(_n, parents, winners)

                M = pm.intermediatePayoffMatrix(B, _T, _n)

                nash = pm.matrixNashEq(M) #get the equilibria
                nash_strats = np.transpose(np.where(nash == True)) #get the strategy profiles
                nash_strats = np.array([np.array(element) for element in nash_strats]) #convert to array

                nash_strats0 = nash_strats[:,0] #extract possible strategies for miner 0
                nash_strats1 = nash_strats[:,1] #extract possible strategies for miner 1

                for element in itertools.product(nash_strats0, nash_strats1): #check if the mask 'nash' is 'False' at any strategy profile in the payoff matrix that could be played. If yes, then we have an example where the conjecture is violated.
                    if nash[element] == False:
                        print("FOUND")
                        return 0

        print(f"checked all situations for t = {t+1} and  T = {_T}") #print status